# vector db 활용 실습

- 주제 : 내용으로 뉴스 카테고리 분석 및 키워드에 맞는 뉴스 내용 출력
1. 적절한 크기의 데이터셋을 구한다.
2. 한국어 임베딩에 적절한 모델을 찾는다.
3. Chroma DB, FAISS, Pinecone 중 원하는 벡터 DB를 이용한다.

In [12]:
import pandas as pd
import numpy as np

In [11]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path='./chroma_db')

collection = client.get_or_create_collection(name='news')

model = SentenceTransformer("jhgan/ko-sroberta-multitask")


In [29]:
df = pd.read_csv('./data/naver.csv')
df

,id,title,author,genre,description,rating,date,completed,age,free,link
0,756056,가난을 등에 업은 소녀,B급달궁 / 오은지,"스토리, 로맨스",흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...,9.11,2020.10.19 00:00,True,전체연령가,True,https://comic.naver.com/webtoon/list?titleId=7...
1,670144,가담항설,랑또,"스토리, 판타지",이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...,9.98,2020.10.07 23:00,True,12세 이용가,True,https://comic.naver.com/webtoon/list?titleId=6...
2,732071,가령의 정체불명 이야기,가령,"옴니버스, 드라마",어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...,9.95,2020.02.03 23:22,True,15세 이용가,True,https://comic.naver.com/webtoon/list?titleId=7...
3,703844,가비지타임,2사장,"스토리, 스포츠","한국 최초 한국형 고교스포츠 웹툰!열혈따윈 개나 줘, 낙오자들 뿐인 농구부의 운명은?",9.96,2022.05.28 22:58,False,전체연령가,False,https://comic.naver.com/webtoon/list?titleId=7...
4,785701,가상&RPG,주다현,"스토리, 판타지","""사람은 좀 죽여도 싸우는건 싫어해요."" 게임에서조차 집 바깥을 나가지 않는 '제리...",9.94,2022.06.01 22:57,False,12세 이용가,False,https://comic.naver.com/webtoon/list?titleId=7...
...,...,...,...,...,...,...,...,...,...,...,...
1851,718018,[드라마원작] 알고있지만,정서,"스토리, 로맨스","""넌 내가 나비라는걸 알고 있잖아.""알면서도 빠지고 싶은 사랑<투명한 동거> 정서 ...",9.97,2019.07.17,True,15세 이용가,True,https://comic.naver.com/webtoon/list?titleId=7...
1852,697679,[드라마원작] 쌉니다 천리...,김규삼,"스토리, 개그","""고객은 왕이 아니다, 직원이 왕이다!""병맛 드라마를 뛰어넘는 미친 상상력의 원작 웹툰",9.98,2018.06.21,True,전체연령가,True,https://comic.naver.com/webtoon/list?titleId=6...
1853,675393,[드라마원작] 한번 더 해요,미티 / 구구,"스토리, 드라마",결혼 8년차 부부. 성대광과 유선영. 대학시절 킹카 퀸카로 CC로 만나 결혼까지 골...,9.91,2018.03.22,True,18세 이용가,True,https://comic.naver.com/webtoon/list?titleId=6...
1854,677536,[드라마원작] 내 ID는 ...,기맹기,"스토리, 로맨스","못생긴 얼굴에서 미인으로 새롭게 태어난 그녀, 강미래! 그런데... 어디서 본 듯 한데?",9.83,2017.12.29,True,12세 이용가,True,https://comic.naver.com/webtoon/list?titleId=6...


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1856 entries, 0 to 1855
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1856 non-null   int64  
 1   title        1856 non-null   object 
 2   author       1856 non-null   object 
 3   genre        1856 non-null   object 
 4   description  1856 non-null   object 
 5   rating       1856 non-null   float64
 6   date         1856 non-null   object 
 7   completed    1856 non-null   bool   
 8   age          1780 non-null   object 
 9   free         1856 non-null   bool   
 10  link         1856 non-null   object 
dtypes: bool(2), float64(1), int64(1), object(7)
memory usage: 134.2+ KB


In [ ]:
# 

doc_des = df['description'][:1000].tolist()
doc_category = df['genre'][:1000].tolist()
doc_headline = df['title'][:1000].tolist()
len(doc_des), len(doc_category), len(doc_headline)

(1000, 1000, 1000)

In [53]:
embeddings = model.encode(doc_des)

for i, (embedding, category, headline) in enumerate(zip(embeddings, doc_category,doc_headline)):
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{'headline': headline ,'category': category, 'description': doc_des[i]}]
    )

Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 4
Insert of existing embedding ID: 4
Add of existing embedding ID: 5
Insert of existing embedding ID: 5
Add of existing embedding ID: 6
Insert of existing embedding ID: 6
Add of existing embedding ID: 7
Insert of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 8
Add of existing embedding ID: 9
Insert of existing embedding ID: 9
Add of existing embedding ID: 10
Insert of existing embedding ID: 10
Add of existing embedding ID: 11
Insert of existing embedding ID: 11
Add of existing embedding ID: 12
Insert of existing embedding ID: 12
Add of existing embedding ID: 13
Insert of existing embedding ID: 13
Add of existing embedding ID: 14
Insert of existing em

In [69]:
def search_webtoon(keyword, top_k=5):
    keyword_embedding = model.encode(keyword).tolist()
    
    results = collection.query(
        query_embeddings=keyword_embedding,
        n_results=top_k
    )
    results = collection.get(
        include=['metadatas']
    )
    
    for result in results['metadatas'][0]:
        print(f"headline: {result['headline']}")
        print(f"Category: {result['category']}")
        print(f"Description: {result['description']}")
        print()


In [70]:
search_webtoon('평범')

TypeError: string indices must be integers